In [10]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data from Excel
file_path = "study_plan.xlsx" # currently a mock-up data is generated..
activities_df = pd.read_excel(file_path, sheet_name="Activities")
availability_df = pd.read_excel(file_path, sheet_name="Week Availability")
semester_info_df = pd.read_excel(file_path, sheet_name="Semester Info")

# Extract semester information
num_weeks = int(semester_info_df["Num Weeks"][0])

# Extract activities and due dates
activities = activities_df["Activity"].tolist()
required_hours = dict(zip(activities_df["Activity"], activities_df["Required Hours"]))
due_dates = dict(zip(activities_df["Activity"], zip(activities_df["Due Week"], activities_df["Due Day"])))

# Days of the week
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert Week Availability to a dictionary
availability = {}
for week in range(1, num_weeks + 1):
    for day in days:
        if (week, day) not in availability:
            availability[(week, day)] = availability_df.loc[week-1, day]

# Initialize the model
model = gp.Model("Study_Plan")

# Decision variables: x[i, week, day] is the study hours for activity i in week 'week' on day 'day'
x = model.addVars(activities, range(1, num_weeks + 1), days, name="study_hours", vtype=GRB.CONTINUOUS)

# Auxiliary variables to capture min/max workload ratios
min_ratio = model.addVars(range(1, num_weeks + 1), days, name="min_ratio", vtype=GRB.CONTINUOUS)
max_ratio = model.addVars(range(1, num_weeks + 1), days, name="max_ratio", vtype=GRB.CONTINUOUS)

# Objective: Minimize the difference between max_ratio and min_ratio across all days
model.setObjective(gp.quicksum(
    max_ratio[week, day] - min_ratio[week, day] 
    for week in range(1, num_weeks + 1) for day in days), GRB.MINIMIZE)

# Constraints for workload ratio per day
for week in range(1, num_weeks + 1):
    for day in days:
        # Total study hours on this day
        total_study_hours = gp.quicksum(x[i, week, day] for i in activities)
        
        # Workload ratio constraints
        model.addConstr(min_ratio[week, day] <= total_study_hours / availability[(week, day)], name=f"min_ratio_constraint_{week}_{day}")
        model.addConstr(max_ratio[week, day] >= total_study_hours / availability[(week, day)], name=f"max_ratio_constraint_{week}_{day}")

# Constraint: Study hours per activity must meet the required hours
for i in activities:
    model.addConstr(gp.quicksum(x[i, week, day] for week in range(1, num_weeks + 1) for day in days) >= required_hours[i], name=f"required_hours_{i}")

# Constraint: Daily study time limit for each week
for week in range(1, num_weeks + 1):
    for day in days:
        model.addConstr(gp.quicksum(x[i, week, day] for i in activities) <= availability[(week, day)], name=f"daily_limit_week{week}_{day}")

# Constraint: Ensure activities are completed by their due dates
for i, (due_week, due_day) in due_dates.items():
    model.addConstr(gp.quicksum(x[i, week, day] for week in range(1, due_week + 1) for day in days) >= required_hours[i], name=f"due_date_{i}")

# Solve the model
model.optimize()


# Check if a solution is found
if model.status == GRB.OPTIMAL:
    # Prepare the study plan in the new format
    study_plan_output = {}

    # Initialize dictionary for each week
    for week in range(1, num_weeks + 1):
        study_plan_output[week] = {}
        for day in days:
            study_plan_output[week][day] = {"Activity": "", "Study Hours": 0}
    
    # Fill the dictionary with the optimized study plan
    for week in range(1, num_weeks + 1):
        for day in days:
            for i in activities:
                hours_allocated = x[i, week, day].x
                if hours_allocated > 0:
                    study_plan_output[week][day] = {"Activity": i, "Study Hours": hours_allocated}
    
    # Create a list of rows for the new Excel format
    output_rows = []
    for week in range(1, num_weeks + 1):
        row = [week]  # First column is the week
        for day in days:
            activity = study_plan_output[week][day]["Activity"]
            study_hours = study_plan_output[week][day]["Study Hours"]
            row.append(activity)
            row.append(study_hours)
        output_rows.append(row)
    
    # Define the columns (day + activity and hours per day)
    columns = ['Week']
    for day in days:
        columns.append(f'{day} Activity')
        columns.append(f'{day} Study Hours')

    # Convert to pandas DataFrame
    study_plan_df = pd.DataFrame(output_rows, columns=columns)

    # Write to Excel file
    output_file = "optimized_study_plan.xlsx"
    study_plan_df.to_excel(output_file, index=False)

    print(f"Study plan successfully saved to {output_file}")

else:
    print("No optimal solution found")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 180 rows, 448 columns and 1694 nonzeros
Model fingerprint: 0xdff1777a
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+01]
Presolve removed 114 rows and 112 columns
Presolve time: 0.01s
Presolved: 66 rows, 336 columns, 798 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.800000e+01   0.000000e+00      0s
      49    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 49 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Study plan successfully saved to optimized_study_plan.xlsx


<ipython-input-10-1db6aa6fac07>:112: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  study_plan_df.to_excel(output_file, index=False)
